In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew
import matplotlib.pyplot as plt
%matplotlib auto

In [2]:
train=pd.read_csv('F:\\study\\ml\\DataSet\\House_Prices\\train.csv')
test=pd.read_csv('F:\\study\\ml\\DataSet\\House_Prices\\test.csv')

In [3]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
train['GarageType'].unique()

array(['Attchd', 'Detchd', 'BuiltIn', 'CarPort', nan, 'Basment', '2Types'],
      dtype=object)

delete outlier data and drop id

In [5]:
plt.scatter(train['GrLivArea'],train['SalePrice'])


In [6]:
train=train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

In [7]:
plt.scatter(train['GrLivArea'],train['SalePrice'])


In [8]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [9]:
train_id=train['Id']
test_id=test['Id']

In [10]:
train.drop('Id',axis=1,inplace=True)
test.drop('Id',axis=1,inplace=True)

- fit norm dist plot

In [11]:
sns.distplot(train['SalePrice'],fit=norm)


- - plot Q-Q 
- double check skew >0

In [12]:
stats.probplot(train['SalePrice'],plot=plt)


((array([-3.3047554 , -3.04752042, -2.90446807, ...,  2.90446807,
          3.04752042,  3.3047554 ]),
  array([ 34900,  35311,  37900, ..., 625000, 745000, 755000], dtype=int64)),
 (74213.25959976624, 180932.91906721535, 0.9320154492892367))

 - log transform target variable

In [13]:
sns.distplot(np.log1p(train['SalePrice']),fit=norm)


In [14]:
sns.distplot(np.log(1/train['SalePrice']),fit=norm)


In [15]:
sns.distplot(np.sqrt(train['SalePrice']),fit=norm)


In [16]:
train['SalePrice'].skew()

1.8812964895244009

In [17]:
train['SalePrice'].kurt()

6.523066888485879

In [18]:
train['SalePrice']=np.log1p(train['SalePrice'])
(mu,sigma)=norm.fit(train['SalePrice'])
stats.probplot(train['SalePrice'],plot=plt)


((array([-3.3047554 , -3.04752042, -2.90446807, ...,  2.90446807,
          3.04752042,  3.3047554 ]),
  array([10.46027076, 10.47197813, 10.54273278, ..., 13.34550853,
         13.52114084, 13.53447435])),
 (0.3985294832980731, 12.024015155682548, 0.9953918721417084))

In [19]:
train['SalePrice'].skew()

0.12157976050304882

In [20]:
train['SalePrice'].kurt()

0.8047507917418972

- features engineing

In [21]:
ntrain=train.shape[0]
ntest=test.shape[0]
y_train=train.SalePrice.values
all_data=pd.concat((train,test)).reset_index(drop=True)
all_data.drop(['SalePrice'],axis=1,inplace=True)
print('all data shape:',all_data.shape)

all data shape: (2917, 79)


- Missing Data

In [22]:
all_data_na=(all_data.isnull().sum()/len(all_data)) * 100
all_data_na=all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)[:30]
missing_data=pd.DataFrame({'Missing Ratio': all_data_na})
missing_data.head(20)

,Missing Ratio
PoolQC,99.691464
MiscFeature,96.400411
Alley,93.212204
Fence,80.425094
FireplaceQu,48.680151
LotFrontage,16.660953
GarageQual,5.450806
GarageCond,5.450806
GarageFinish,5.450806
GarageYrBlt,5.450806


In [23]:
plt.subplots(figsize=(12,10))
plt.xticks(rotation='90')
sns.barplot(x=all_data_na.index,y=all_data_na)
plt.xlabel('Features',fontsize=15)
plt.ylabel('Precent of missing value',fontsize=15)
plt.title('Precent of miss data by feature',fontsize=15)


Text(0.5,1,'Precent of miss data by feature')

- Data Correlation

In [24]:
corrmat=train.corr()
sns.heatmap(corrmat,vmax=0.9,square=True)

- imputing missing values

In [25]:
all_data['LotFrontage'].head()

0    65.0
1    80.0
2    68.0
3    60.0
4    84.0
Name: LotFrontage, dtype: float64

In [26]:
all_data['Neighborhood'].head()

0    CollgCr
1    Veenker
2    CollgCr
3    Crawfor
4    NoRidge
Name: Neighborhood, dtype: object

In [27]:
for i in all_data.groupby('Neighborhood')['LotFrontage']:
    print(i)

('Blmngtn', 219     43.0
229     43.0
385     43.0
443     53.0
465      NaN
558      NaN
596     53.0
638     53.0
789     43.0
850      NaN
981     43.0
1003    43.0
1018    43.0
1022    43.0
1125    53.0
1392    53.0
1413    51.0
1481    43.0
1683    43.0
1982     NaN
1983     NaN
1984    53.0
2318     NaN
2319    53.0
2669    43.0
2670     NaN
2671     NaN
2672    43.0
Name: LotFrontage, dtype: float64)
('Blueste', 598     24.0
955     24.0
1596    35.0
1597    35.0
1905    24.0
1906    24.0
1907    24.0
2207    24.0
2567    24.0
2569    35.0
Name: LotFrontage, dtype: float64)
('BrDale', 225     21.0
227     21.0
232     21.0
235     21.0
363     21.0
430     21.0
432     24.0
500     21.0
654     21.0
836     21.0
1028    21.0
1103    24.0
1218    21.0
1290    21.0
1332    24.0
1376    21.0
1469    21.0
1470    21.0
1649    21.0
1650    21.0
1959    21.0
1960    21.0
1961    21.0
2280    21.0
2281    21.0
2640    24.0
2641    24.0
2642    21.0
2643    21.0
2644    21.0
Name: LotFr

In [28]:
all_data['PoolQC']=all_data['PoolQC'].fillna('None')
all_data['MiscFeature']=all_data['MiscFeature'].fillna('None')
all_data['Alley']=all_data['Alley'].fillna('None')
all_data['Fence']=all_data['Fence'].fillna('None')
all_data['FireplaceQu']=all_data['FireplaceQu'].fillna('None')
all_data['LotFrontage']=all_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x:x.fillna(x.median()))

In [29]:
for col in ['GarageType','GarageFinish','GarageQual','GarageCond']:
    all_data[col]=all_data[col].fillna('None')

In [30]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)

In [31]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)

In [32]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')

In [33]:
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

In [34]:
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])

In [35]:
all_data = all_data.drop(['Utilities'], axis=1)

In [36]:
all_data["Functional"] = all_data["Functional"].fillna("Typ")

In [37]:
all_data['Electrical']=all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])

In [38]:
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])

In [39]:
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])

In [40]:
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

In [41]:
all_data_na=(all_data.isnull().sum())/len(all_data) * 100
all_data_na=all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)
miss_data=pd.DataFrame({'Missing Ratio':all_data_na})
miss_data.head()

,Missing Ratio


In [42]:
all_data['MSSubClass'].dtype

dtype('int64')

In [43]:
all_data['MSSubClass']=all_data['MSSubClass'].apply(str)

In [44]:
all_data['MSSubClass'].dtype

dtype('O')

In [45]:
all_data['OverallCond'] = all_data['OverallCond'].astype(str)

In [46]:
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [47]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond',
        'ExterQual', 'ExterCond', 'HeatingQC', 'PoolQC', 'KitchenQual',
        'BsmtFinType1', 'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure',
        'GarageFinish', 'LandSlope', 'LotShape', 'PavedDrive', 'Street',
        'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 'YrSold', 'MoSold')
for c in cols:
    lbl=LabelEncoder()
    lbl.fit(list(all_data[c].values))
    all_data[c]=lbl.transform(list(all_data[c].values))

In [48]:
all_data.shape

(2917, 78)

In [49]:
all_data['TotalSF']=all_data['TotalBsmtSF']+all_data['1stFlrSF']+all_data['2ndFlrSF']

- skewed features

In [50]:
numeric_feats=all_data.dtypes[all_data.dtypes != 'object'].index
skewed_feats=all_data[numeric_feats].apply(lambda x : skew(x.dropna())).sort_values(ascending=False)
skewness=pd.DataFrame({'Skew':skewed_feats,'kurt':all_data[numeric_feats].kurt()})
skewness

,Skew,kurt
1stFlrSF,1.257286,5.075293
2ndFlrSF,0.861556,-0.424185
3SsnPorch,11.372080,149.304586
Alley,-0.651705,11.682122
BedroomAbvGr,0.326568,1.938367
BsmtCond,-2.861271,6.739601
BsmtExposure,-1.117321,-0.229041
BsmtFinSF1,0.980645,1.427134
BsmtFinSF2,4.144503,18.828682
BsmtFinType1,0.083641,-1.509954


- get abs(skew ) > 0.75

In [51]:
skewness

,Skew,kurt
1stFlrSF,1.257286,5.075293
2ndFlrSF,0.861556,-0.424185
3SsnPorch,11.372080,149.304586
Alley,-0.651705,11.682122
BedroomAbvGr,0.326568,1.938367
BsmtCond,-2.861271,6.739601
BsmtExposure,-1.117321,-0.229041
BsmtFinSF1,0.980645,1.427134
BsmtFinSF2,4.144503,18.828682
BsmtFinType1,0.083641,-1.509954


In [55]:
skewness=skewness[abs(skewness['Skew'])>0.75]

In [56]:
from scipy.special import boxcox1p
from scipy.stats import boxcox,boxcox_normmax

In [57]:
for i in range(len(skewness)):
    print(skewness.index[i],'-----skew:',skewness.values[i],'-----lm:',boxcox_normmax(all_data[skewness.index[i]]+1),'kurt :',all_data[skewness.index[i]].kurt())
    

1stFlrSF -----skew: [1.25728598 5.07529277] -----lm: -0.03890776331837051 kurt : 5.075292766543552
2ndFlrSF -----skew: [ 0.86155552 -0.42418458] -----lm: 1.0317215975523517 kurt : -0.4241845756906404
3SsnPorch -----skew: [ 11.37207993 149.30458575] -----lm: 0.0701437654661614 kurt : 149.3045857509312
BsmtCond -----skew: [-2.86127142  6.73960113] -----lm: 1.6190843743095362 kurt : 6.739601130805046
BsmtExposure -----skew: [-1.11732123 -0.22904084] -----lm: 2.6925156702400868 kurt : -0.22904083970559252
BsmtFinSF1 -----skew: [0.98064459 1.42713428] -----lm: 0.7060356810946441 kurt : 1.4271342763364538
BsmtFinSF2 -----skew: [ 4.14450336 18.82868204] -----lm: 0.16704477665499903 kurt : 18.82868204421994
BsmtFinType2 -----skew: [-3.04297932  8.49298479] -----lm: 4.8223572696308326 kurt : 8.492984788152512
BsmtHalfBath -----skew: [ 3.92999597 14.84807931] -----lm: 0.07264586855126172 kurt : 14.848079309450544
BsmtUnfSF -----skew: [0.91968821 0.40304222] -----lm: 0.5806040340318408 kurt : 0.4

- - change fixed lam=0.15 to different lam

In [58]:
skewed_feats=skewness.index
for feat in skewed_feats:
    lm=boxcox_normmax(all_data[feat]+1)
    all_data[feat]=boxcox1p(all_data[feat],lm)
    print(feat,' skew : ', all_data[feat].skew(),'lm: ',lm,'---------','kurt : ',all_data[feat].kurt())

1stFlrSF  skew :  -0.007826213842105453 lm:  -0.03890776331837051 --------- kurt :  0.03729062705093167
2ndFlrSF  skew :  0.8959137713038372 lm:  1.0317215975523517 --------- kurt :  -0.3063433190568423
3SsnPorch  skew :  8.866604745584935 lm:  0.0701437654661614 --------- kurt :  77.44835335490302
BsmtCond  skew :  -2.7211286231667544 lm:  1.6190843743095362 --------- kurt :  5.713869342020861
BsmtExposure  skew :  -0.45328966252678377 lm:  2.6925156702400868 --------- kurt :  -0.37707525935216335
BsmtFinSF1  skew :  0.36617452563554187 lm:  0.7060356810946441 --------- kurt :  -0.750143358453887
BsmtFinSF2  skew :  2.5796555076069794 lm:  0.16704477665499903 --------- kurt :  5.035592905404351
BsmtFinType2  skew :  -2.208379138933882 lm:  4.8223572696308326 --------- kurt :  3.12639681834567
BsmtHalfBath  skew :  3.781379040015204 lm:  0.07264586855126172 --------- kurt :  12.67040103504985
BsmtUnfSF  skew :  0.05422301845805017 lm:  0.5806040340318408 --------- kurt :  -0.4550187108

In [59]:
for i in range(len(skewness)):
    print(skewness.index[i],'-----skew:',skewness.values[i],'-----lm:',boxcox_normmax(all_data[skewness.index[i]]+1))

1stFlrSF -----skew: [1.25728598 5.07529277] -----lm: 1.0143825000402447
2ndFlrSF -----skew: [ 0.86155552 -0.42418458] -----lm: 1.0000044937889796
3SsnPorch -----skew: [ 11.37207993 149.30458575] -----lm: 0.9274480468821471
BsmtCond -----skew: [-2.86127142  6.73960113] -----lm: 1.0127787899006315
BsmtExposure -----skew: [-1.11732123 -0.22904084] -----lm: 1.0176715012600075
BsmtFinSF1 -----skew: [0.98064459 1.42713428] -----lm: 0.999064837484709
BsmtFinSF2 -----skew: [ 4.14450336 18.82868204] -----lm: 0.9644408839557721
BsmtFinType2 -----skew: [-3.04297932  8.49298479] -----lm: 1.002778709827105
BsmtHalfBath -----skew: [ 3.92999597 14.84807931] -----lm: 1.0000003768467427
BsmtUnfSF -----skew: [0.91968821 0.40304222] -----lm: 0.9997207341790258
CentralAir -----skew: [-3.45755483  9.97382944] -----lm: -4.472157577133573
EnclosedPorch -----skew: [ 4.00234409 28.35803934] -----lm: 0.9851771165653529
ExterCond -----skew: [-2.49648923  5.08711944] -----lm: 1.000754159974327
ExterQual -----skew

D:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3018: RuntimeWarning: invalid value encountered in subtract
  xm, ym = x - mx, y - my
D:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:5277: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0
D:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:5438: RuntimeWarning: overflow encountered in multiply
  return np.sum(a*a, axis)


In [60]:
for feat in skewed_feats:
    lm=boxcox_normmax(all_data[feat]+2)
    all_data[feat]=boxcox1p(all_data[feat],lm)
    print(feat,' skew : ', all_data[feat].skew(),'lm: ',lm,'---------','kurt : ',all_data[feat].kurt())

1stFlrSF  skew :  -0.006255450675372226 lm:  1.0149866462920527 --------- kurt :  0.03729039859086081
2ndFlrSF  skew :  0.8960689587616563 lm:  1.000138790973803 --------- kurt :  -0.30579645624251306
3SsnPorch  skew :  8.85458217855458 lm:  0.9322067533800326 --------- kurt :  77.17198706197541
BsmtCond  skew :  -2.7114109482897932 lm:  1.041947512736244 --------- kurt :  5.644925697461173
BsmtExposure  skew :  -0.408613782100056 lm:  1.0365004521797931 --------- kurt :  -0.3175371936502782
BsmtFinSF1  skew :  0.36774990366892424 lm:  1.001159858178507 --------- kurt :  -0.7467721388666195
BsmtFinSF2  skew :  2.5717408758206513 lm:  0.9717535112144385 --------- kurt :  4.981901956197101
BsmtFinType2  skew :  -2.2065980105723932 lm:  1.0055212251182786 --------- kurt :  3.116743934963151
BsmtHalfBath  skew :  3.781379086112784 lm:  1.0000007795300778 --------- kurt :  12.670401652503969
BsmtUnfSF  skew :  0.05437903181079886 lm:  1.0000976616522792 --------- kurt :  -0.4550359210379868

D:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3018: RuntimeWarning: invalid value encountered in subtract
  xm, ym = x - mx, y - my
D:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:5277: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0
D:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:5438: RuntimeWarning: overflow encountered in multiply
  return np.sum(a*a, axis)


In [ ]:
all_data.head()

In [ ]:
all_data=pd.get_dummies(all_data)
all_data.shape

In [ ]:
train=all_data[:ntrain]
test=all_data[ntrain:]

#### Model

In [ ]:
from sklearn.linear_model import ElasticNet,Lasso,BayesianRidge,LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator,TransformerMixin,RegressorMixin,clone
from sklearn.model_selection import KFold,cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [ ]:
n_folds=5

def rmsle_cv(model):
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(train.values)
    rmse=np.sqrt(-cross_val_score(model,train.values,y_train,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [ ]:
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))

In [ ]:
ENet=make_pipeline(RobustScaler(),ElasticNet(alpha=0.0005,l1_ratio=0.9,random_state=3))

In [ ]:
KRR=KernelRidge(alpha=0.6,kernel='polynomial',degree=2,coef0=2.5)

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000,
                                   learning_rate=0.05,
                                   max_depth=4,
                                   max_features='sqrt',
                                   min_samples_leaf=15,
                                   min_samples_split=10,
                                   loss='huber',
                                   random_state=5)

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [ ]:
score=rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

In [ ]:
0.7*0.7*0.3*3

In [ ]:
0.7*0.7*0.7*3

### Stacked Model
- Simple stacking approach: average base models

In [ ]:
class 

In [71]:
str1=''
for i in range(69,91):
    str1+=chr(i)+'5&","&'
str1

'E5&","&F5&","&G5&","&H5&","&I5&","&J5&","&K5&","&L5&","&M5&","&N5&","&O5&","&P5&","&Q5&","&R5&","&S5&","&T5&","&U5&","&V5&","&W5&","&X5&","&Y5&","&Z5&","&'

In [ ]:
'E5&","F5&","G5&","H5&","I5&","J5&","K5&","L5&","M5&","N5&","O5&","P5&","Q5&","R5&","S5&","T5&","U5&","V5&","W5&","X5&","Y5&","Z5&","AA5&","AB5&","AC5'